# Segmenting and Clustering Neighborhoods in Toronto

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1.  [Scraping Wikipedia Dataset](#item1)

2.  [Add Longitude and Latitude](#item2)

3.  [Explore Neighborhoods in Toronto City](#item3)

    </font>
    </div>

In [1]:
## Libraries

from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd
import json
from pandas.io.json import json_normalize
import numpy as np
import matplotlib.pyplot as plt

## 1. Scraping Wikipedia Dataset

In [2]:
# send the GET request
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'lxml')

# create three lists to store table data
postalCodeList = []
boroughList = []
neighborhoodList = []

# append the data into the respective lists
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text.rstrip('\n'))
        boroughList.append(cells[1].text.rstrip('\n'))
        neighborhoodList.append(cells[2].text.rstrip('\n'))

### Dataframe construction rules

* The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
* Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
* More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11  in the above table.
* If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.
* Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
* In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [3]:
# create a new DataFrame from the three lists
toronto_df = pd.DataFrame({"Postal Code": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})

toronto_df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [4]:
# drop cells with Borough not assigned
toronto_dropped = toronto_df[toronto_df.Borough != 'Not assigned'].reset_index(drop=True)
toronto_dropped.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [5]:
# group multiple neighborhoods having same postal code
toronto_group = toronto_dropped.groupby(['Postal Code', 'Borough'], as_index=False).agg(lambda x: ", ".join(x))
toronto_group['Neighborhood'] = toronto_group['Neighborhood'].str.replace('/', ',')
toronto_group.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [6]:
# rows of dataframe
toronto_group.shape

(103, 3)

## 2. Add Longitude and Latitude

#### Adding Lattitude and Longitude to the dataframe

In [7]:
# !pip install geocoder
# import geocoder

# # create two lists to accommodate longitude and latitude
# latitude = [] 
# longitude = []

# # iterate through each postal code in dataframe
# for postal_code in toronto_group['Postal Code'].tolist():
#     g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#     lat_lng_coords = g.latlng
#     latitude.append(lat_lng_coords[0])
#     longitude.append(lat_lng_coords[1])

In [8]:
# download the data of latitude and longitude: link provided by >>>>coursera
# !wget http://cocl.us/Geospatial_data

# read Geospatial_data
latlon = pd.read_csv('Geospatial_data')

# merge two dataframes using inner join on Postal Code column
df = pd.merge(toronto_group, latlon, how='inner', on ='Postal Code')
    
print(df.shape)
df.head(10)

(103, 5)


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


## 3. Explore Neighborhoods in Toronto City

In [9]:
print('The dataframe has {} boroughs'.format(len(df['Borough'].unique())))

The dataframe has 10 boroughs


In [10]:
!pip install geopy
from geopy.geocoders import Nominatim
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

In [11]:
# Create a map of Toronto with neighbourhood superimposed on it

# create map of New York using latitude and longitude values
address = 'Toronto , Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


In [12]:
df['Borough'].unique()

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       'Mississauga', 'Etobicoke'], dtype=object)

### Segment and cluster only the neighborhoods in Downtown Toronto

In [13]:
downtown_data = df[df['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
downtown_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [14]:
address = 'Downtown Toronto ,Toronto, Ontario'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map of New York using latitude and longitude values
map_dwontown = folium.Map(location=[latitude, longitude], zoom_start= 11)

# add markers to map
for lat, lng, borough, neighborhood in zip(downtown_data['Latitude'], downtown_data['Longitude'], 
                                           downtown_data['Borough'], downtown_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_dwontown)  
    
map_dwontown

In [34]:
# @hidden_cell

CLIENT_ID = '----' # your Foursquare ID
CLIENT_SECRET = '----' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


In [16]:
#definig the latitude and longitude using above dataframe
lat = downtown_data.loc[0, 'Latitude'] # neighborhood latitude value
lon = downtown_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = downtown_data.loc[0, 'Neighborhood'] # neighborhood name
print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, lat, lon))

LIMIT = 100
radius =1000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET, VERSION, lat,lon, radius, LIMIT)

# gettig the venues data form Forsquare API in json format
results = requests.get(url).json()
results

Latitude and longitude values of Rosedale are 43.6795626, -79.37752940000001.


{'meta': {'code': 200, 'requestId': '5fdbc5177b92187a3b64c67b'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Rosedale',
  'headerFullLocation': 'Rosedale, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 22,
  'suggestedBounds': {'ne': {'lat': 43.68856260900001,
    'lng': -79.36510816548741},
   'sw': {'lat': 43.670562590999985, 'lng': -79.38995063451262}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4adcb343f964a520e32e21e3',
       'name': 'Summerhill Market',
       'location': {'address': '446 Summerhill Ave',
        'crossStreet': 'btwn. MacLennan Ave. and Glen Rd.',
        'lat': 43.68626482142425,
        'lng': -79.37545823237794,
      

In [18]:
venues = results['response']['groups'][0]['items']
venues_df = json_normalize(venues) # flatten JSON
venues_df.head(5)

/home/yi-ting/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.crossStreet,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,...,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.location.neighborhood,venue.venuePage.id
0,e-0-4adcb343f964a520e32e21e3-0,0,"[{'summary': 'This spot is popular', 'type': '...",4adcb343f964a520e32e21e3,Summerhill Market,446 Summerhill Ave,btwn. MacLennan Ave. and Glen Rd.,43.686265,-79.375458,"[{'label': 'display', 'lat': 43.68626482142425...",...,CA,Toronto,ON,Canada,[446 Summerhill Ave (btwn. MacLennan Ave. and ...,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",0,[],NaN,NaN
1,e-0-4ad88fa9f964a520511221e3-1,0,"[{'summary': 'This spot is popular', 'type': '...",4ad88fa9f964a520511221e3,Black Camel,4 Crescent Rd,Yonge,43.677016,-79.389367,"[{'label': 'display', 'lat': 43.67701587839681...",...,CA,Toronto,ON,Canada,"[4 Crescent Rd (Yonge), Toronto ON M4W 1S9, Ca...","[{'id': '4bf58dd8d48988d1df931735', 'name': 'B...",0,[],NaN,NaN
2,e-0-4ba5156cf964a520b6da38e3-2,0,"[{'summary': 'This spot is popular', 'type': '...",4ba5156cf964a520b6da38e3,Toronto Lawn Tennis Club,44 Price St.,at Yonge St.,43.680667,-79.388559,"[{'label': 'display', 'lat': 43.68066694338894...",...,CA,Toronto,ON,Canada,"[44 Price St. (at Yonge St.), Toronto ON, Canada]","[{'id': '4f4528bc4b90abdf24c9de85', 'name': 'A...",0,[],NaN,NaN
3,e-0-5c6964c1a87921002c41f5f5-3,0,"[{'summary': 'This spot is popular', 'type': '...",5c6964c1a87921002c41f5f5,Maison Selby,592 Sherbourne St,Selby St,43.671232,-79.376618,"[{'label': 'display', 'lat': 43.6712319213305,...",...,CA,Toronto,ON,Canada,"[592 Sherbourne St (Selby St), Toronto ON M4X ...","[{'id': '52e81612bcbc57f1066b79f1', 'name': 'B...",0,[],NaN,NaN
4,e-0-54049701498ede7ef8e77dfe-4,0,"[{'summary': 'This spot is popular', 'type': '...",54049701498ede7ef8e77dfe,Pie Squared,366 Bloor St E,Sherbourne St.,43.672143,-79.377856,"[{'label': 'display', 'lat': 43.67214294983148...",...,CA,Toronto,ON,Canada,"[366 Bloor St E (Sherbourne St.), Toronto ON M...","[{'id': '52e81612bcbc57f1066b7a0a', 'name': 'P...",0,[],Downtown Toronto,NaN


- We can see there are many details about the venues near first neighbourhood of Downtown Toronto.
- But we're interested in only the venue name, categories, loaction (latitde and longitude).
- Let's take only those columns which are needed.

In [19]:
# filter columns
cols = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
venues_df = venues_df.loc[:, cols]

# to show whole text in columns in dataframe python: https://stackoverflow.com/a/25352191
pd.set_option("display.max_colwidth" , -1)
venues_df.head()

/home/yi-ting/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Summerhill Market,"[{'id': '4bf58dd8d48988d118951735', 'name': 'Grocery Store', 'pluralName': 'Grocery Stores', 'shortName': 'Grocery Store', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_', 'suffix': '.png'}, 'primary': True}]",43.686265,-79.375458
1,Black Camel,"[{'id': '4bf58dd8d48988d1df931735', 'name': 'BBQ Joint', 'pluralName': 'BBQ Joints', 'shortName': 'BBQ', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bbqalt_', 'suffix': '.png'}, 'primary': True}]",43.677016,-79.389367
2,Toronto Lawn Tennis Club,"[{'id': '4f4528bc4b90abdf24c9de85', 'name': 'Athletics & Sports', 'pluralName': 'Athletics & Sports', 'shortName': 'Athletics & Sports', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/sports_outdoors_', 'suffix': '.png'}, 'primary': True}]",43.680667,-79.388559
3,Maison Selby,"[{'id': '52e81612bcbc57f1066b79f1', 'name': 'Bistro', 'pluralName': 'Bistros', 'shortName': 'Bistro', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_', 'suffix': '.png'}, 'primary': True}]",43.671232,-79.376618
4,Pie Squared,"[{'id': '52e81612bcbc57f1066b7a0a', 'name': 'Pie Shop', 'pluralName': 'Pie Shops', 'shortName': 'Pie Shop', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pieshop_', 'suffix': '.png'}, 'primary': True}]",43.672143,-79.377856


In [20]:
# reset set option dataframe: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.reset_option.html
pd.reset_option('display.max_colwidth')

- Name, location (lat and lng) looks good but we still need to etract the catgories.
- Categories is in venue.categories[0]['name']
- everything looks good except the names of column let's change the name of columns

In [21]:
venues_df['venue.categories'] = venues_df.apply(lambda x: x['venue.categories'][0]['name'], axis=1)

# clean columns
venues_df.columns = [col.split(".")[-1] for col in venues_df.columns]
print('{} Venues are returned for: {}'.format(venues_df.shape[0], neighborhood_name))
venues_df.head()

22 Venues are returned for: Rosedale


,name,categories,lat,lng
0,Summerhill Market,Grocery Store,43.686265,-79.375458
1,Black Camel,BBQ Joint,43.677016,-79.389367
2,Toronto Lawn Tennis Club,Athletics & Sports,43.680667,-79.388559
3,Maison Selby,Bistro,43.671232,-79.376618
4,Pie Squared,Pie Shop,43.672143,-79.377856


In [22]:
# define function to repeat the same process to all the neighborhoods in Manhattan

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
print('Finding the near by venues of: ')
downtown_venues = getNearbyVenues(names=downtown_data['Neighborhood'],
                                   latitudes=downtown_data['Latitude'],
                                   longitudes=downtown_data['Longitude']
                                  )

Finding the near by venues of: 
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Queen's Park, Ontario Provincial Government


In [24]:
print(downtown_venues.shape)
downtown_venues.head()

(1248, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
1,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
2,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
3,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail
4,"St. James Town, Cabbagetown",43.667967,-79.367675,Cranberries,43.667843,-79.369407,Diner


In [25]:
print('There are {} uniques categories.'.format(len(downtown_venues['Venue Category'].unique())))
print('\n\nVenues returned for each neighbourhood: ')
downtown_venues.groupby('Neighborhood')['Venue'].count()

There are 210 uniques categories.


Venues returned for each neighbourhood: 


Neighborhood
Berczy Park                                                                                                    55
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport     16
Central Bay Street                                                                                             68
Christie                                                                                                       16
Church and Wellesley                                                                                           75
Commerce Court, Victoria Hotel                                                                                100
First Canadian Place, Underground city                                                                        100
Garden District, Ryerson                                                                                      100
Harbourfront East, Union Station, Toronto Islands                          

### analyze each neighbourhood

In [26]:
# one hot encoding of venue categories columns:https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html
downtown_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix= "", prefix_sep= " ")

# # add neighborhood column back to dataframe
downtown_onehot['Neighbourhood'] = downtown_venues['Neighborhood'] 
# move neighborhood column to the first column
fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
downtown_onehot = downtown_onehot[fixed_columns]
print(downtown_onehot.shape)
downtown_onehot.head()

(1248, 211)


,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,"St. James Town, Cabbagetown",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
downtown_grouped = downtown_onehot.groupby('Neighbourhood').mean().reset_index()
print(downtown_grouped.shape)
downtown_grouped.head()

(19, 211)


,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.018182,0.0,0.000000,0.000000,0.000000
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0625,0.0625,0.0625,0.125,0.125,0.0625,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000
2,Central Bay Street,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.014706,0.0,0.000000,0.014706,0.014706
3,Christie,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000
4,Church and Wellesley,0.013333,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.013333,0.0,...,0.013333,0.013333,0.0,0.0,0.0,0.000000,0.0,0.013333,0.000000,0.026667


### print each neighborhood along with the top 5 most common venues

In [28]:
num_top_venues = 5

for hood in downtown_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = downtown_grouped[downtown_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    
    #first rows is not usefull it has only Neighbourhood and Neighbourhood-name, so drop this row
    temp = temp.iloc[1:]
    temp['freq'] = round(temp['freq'].astype(float),2)# converting into float type and # taking round values
    temp = temp.sort_values('freq', ascending=False).reset_index(drop=True) # sorting the dataframe by 'freq' in decreasing order
    print(temp.head(num_top_venues))
    print('\n')

----Berczy Park----
                 venue  freq
0          Coffee Shop  0.09
1         Cocktail Bar  0.04
2             Beer Bar  0.04
3       Farmers Market  0.04
4   Seafood Restaurant  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0    Airport Lounge  0.12
1   Airport Service  0.12
2   Harbor / Marina  0.06
3           Airport  0.06
4          Boutique  0.06


----Central Bay Street----
                 venue  freq
0          Coffee Shop  0.18
1                 Café  0.06
2       Sandwich Place  0.04
3   Italian Restaurant  0.04
4      Thai Restaurant  0.03


----Christie----
            venue  freq
0   Grocery Store  0.25
1            Café  0.19
2            Park  0.12
3      Restaurant  0.06
4      Baby Store  0.06


----Church and Wellesley----
                  venue  freq
0           Coffee Shop  0.09
1   Japanese Restaurant  0.05
2      Sushi Restaurant  0.05
3               

### put Top venues into a pandas dataframe

In [29]:
# function
def return_most_common_venues(row, num_top_venues):
    row = row.iloc[1:]
    row_sorted = row.sort_values(ascending=False)
    
    return row_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Neighbourhood'] = downtown_grouped['Neighbourhood']

for ind in np.arange(downtown_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cheese Shop,Farmers Market,Cocktail Bar,Restaurant,Seafood Restaurant,Bakery,Beer Bar,Gourmet Shop,Basketball Stadium
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Boat or Ferry,Boutique,Plane,Rental Car Location,Coffee Shop,Harbor / Marina,Sculpture Garden,Airport Terminal
2,Central Bay Street,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Bubble Tea Shop,Salad Place,Japanese Restaurant,Department Store,Thai Restaurant,Burger Joint
3,Christie,Grocery Store,Café,Park,Candy Store,Restaurant,Italian Restaurant,Baby Store,Athletics & Sports,Nightclub,Coffee Shop
4,Church and Wellesley,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Yoga Studio,Men's Store,Mediterranean Restaurant,Hotel,Pub


### Clustering

In [30]:
k = 5

X = downtown_grouped.drop('Neighbourhood', axis = 1)

# run k-means clustering
kmeans = KMeans(n_clusters = k, random_state=0)
kmeans.fit(X)

KMeans(n_clusters=5, random_state=0)

In [31]:
# add clustering labels
venues_sorted['Cluster_Labels']=  kmeans.labels_

downtown_toronto_merged = downtown_data
# merge top venues_sorted with toronto_data
downtown_toronto_merged = downtown_toronto_merged.join(venues_sorted.set_index('Neighbourhood'), on='Neighborhood')

downtown_toronto_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster_Labels
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,Park,Trail,Playground,Dance Studio,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,1
1,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675,Coffee Shop,Pizza Place,Café,Restaurant,Chinese Restaurant,Italian Restaurant,Market,Bakery,Park,Pub,2
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Yoga Studio,Men's Store,Mediterranean Restaurant,Hotel,Pub,2
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,Coffee Shop,Pub,Bakery,Park,Breakfast Spot,Theater,Café,Yoga Studio,Event Space,Performing Arts Venue,0
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Bubble Tea Shop,Japanese Restaurant,Ramen Restaurant,Pizza Place,Bookstore,Middle Eastern Restaurant,2


In [32]:
import matplotlib

# create map
map_clusterd = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [matplotlib.colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_toronto_merged['Latitude'], downtown_toronto_merged['Longitude'],
                                  downtown_toronto_merged['Neighborhood'], downtown_toronto_merged['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusterd)
       
map_clusterd

In [33]:
# function defined to do the work described above, so it can easily be applied on other borough

# def explore_borough(b, n, cluster_k):
#     new_df = df[df['Borough'] == b].reset_index(drop = True)
#     print(new_df.shape)

#     address = b+' ,Toronto, Ontario'
#     geolocator = Nominatim(user_agent="ny_explorer")
#     location = geolocator.geocode(address)
#     latitude = location.latitude
#     longitude = location.longitude

#     venues =  get_near_by_venues(names = new_df['Neighbourhood'],latitudes = new_df['Latitude'], longitudes = new_df['Longitude'])

#     onehot_df = pd.get_dummies(venues[['Venue Category']], prefix= "", prefix_sep= " ")

#     # # add neighborhood column back to dataframe
#     onehot_df['Neighbourhood'] = new_df['Neighbourhood']
#     # move neighborhood column to the first column
#     fixed_columns = [onehot_df.columns[-1]] + list(onehot_df.columns[:-1])
#     onehot_df = onehot_df[fixed_columns]
#     onehot_df_grouped = onehot_df.groupby('Neighbourhood').mean().reset_index()

#     num_top_venues = 10

#     indicators = ['st', 'nd', 'rd']

#     # create columns according to number of top venues
#     columns = ['Neighbourhood']
#     for ind in np.arange(num_top_venues):
#         try:
#             columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
#         except:
#             columns.append('{}th Most Common Venue'.format(ind+1))

#     # create a new dataframe
#     venues_sorted = pd.DataFrame(columns=columns)
#     venues_sorted['Neighbourhood'] = onehot_df_grouped['Neighbourhood']

#     for ind in np.arange(onehot_df_grouped.shape[0]):
#         venues_sorted.iloc[ind, 1:] = return_most_common_venues(onehot_df_grouped.iloc[ind, :], num_top_venues)

#     k = cluster_k
#     X = onehot_df_grouped.drop('Neighbourhood', axis = 1)

#     # run k-means clustering
#     kmeans = KMeans(n_clusters = k, random_state=0)
#     kmeans.fit(X)

#     # add clustering labels
#     venues_sorted['Cluster_Labels']=  kmeans.labels_

#     merged_df = new_df
#     # merge top venues_sorted with toronto_data
#     merged_df = merged_df.join(venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
    
#     # create map
#     borough_map = folium.Map(location=[latitude, longitude], zoom_start=11)

#     # add markers to the map
#     for lat, lon, poi, cluster in zip(merged_df['Latitude'], merged_df['Longitude'], merged_df['Neighbourhood'], merged_df['Cluster_Labels']):
#         label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
#         folium.CircleMarker(
#             [lat, lon],
#             radius=5,
#             popup=label,
#             color=rainbow[cluster-1],
#             fill=True,
#             fill_color=rainbow[cluster-1],
#             fill_opacity=0.7).add_to(borough_map)

#     return borough_map, merged_df

# map1, data = explore_borough(b = 'Scarborough', n = 10, cluster_k = 3)